# Timing Tests of `fastnumbers` Functions Compared to Equivalent Solutions

In order for you to see the benefit of `fastnumbers`, some timings are collected below for comparison to equivalent python implementations. The numbers may change depending on the machine you are on or the Python version you are using.

Feel free to download this Jupyter Notebook and run the tests yourself to see how `fastnumbers` performs on your machine (it takes about 1-2 minutes total).

**This notebook contains timing results for Python 3.6.**

#### Some notes about the data

 - Each test is the time it takes for the function to run 100,000 times on a given input.
 - Each test is repeated either 5 or 100 times, and the mean ± standard deviation is reported.
 - The fastest time is shown in **bold**
 - The timing results for the pure-Python functions include about 10-15 ms of "function call overhead"; the `fastnumbers` functions do not suffer from as much overhead because they are C-extensions.
 - Python version-dependent behaviors:
   - **Python 2.7** has a particularly slow `int` function, so the `fastnumbers` speedup is much larger on Python 2.7 than Python 3.x
   - **Python >=3.6** is slightly slower in general than previous versions because underscores are now allowed in floats and integers which makes parsing take a bit longer due to the extra logic.
   
#### Notes about the `Timing` class below

The timing runner class is implemented below, and this is used in all the tests to perform the actual timing tests in the sections below. In general you can skip this implementation, but of note is the `THINGS_TO_TIME` tuple, which contains the values that are passed to the functions to type the various input types.

In [1]:
from __future__ import print_function, division
import re
import math
import timeit
from IPython.display import Markdown, display, clear_output

class Timer(object):
    """Class to time functions and make pretty tables of the output."""
    
    # This is a list of all the things we will time with an associated label.
    THINGS_TO_TIME = (
        ('not_a_number', 'Non-number String'),
        ('-41053', 'Int String'),
        ('35892482945872302493947939485729', 'Large Int String'),
        ('-41053.543034e34', 'Float String'),
        ('-41053.543028758302e256', 'Large Float String'),
        (-41053, 'Int'),
        (-41053.543028758302e100, 'Float'),
    )

    # Formatting strings.
    FUNCTION_CALL_FMT = '{}({!r})'
    
    def __init__(self, title):
        display(Markdown('### ' + title))
        self.functions = []
    
    def add_function(self, func, label, setup='pass'):
        """Add a function to be timed and compared."""
        self.functions.append((func, setup, label))

    def time_functions(self, repeat=5):
        """Time all the given functions against all input then display results."""

        # Collect the function labels to make the header of this table.
        # Show that the units are seconds for each.
        function_labels = [label + ' (ms)' for _, _, label in self.functions]
        
        # Construct the table strings, formatted in Markdown.
        # Store each line as a string element in a list.
        # This portion here is the table header only for now.
        table = Table()
        table.add_header('Input type', *function_labels)
        
        # For each value, time each function and collect the results.
        for value, value_label in self.THINGS_TO_TIME:
            row = []
            for func, setup, _ in self.functions:
                call = self.FUNCTION_CALL_FMT.format(func, value)
                try:
                    row.append(self._timeit(call, setup, repeat))
                except (ValueError, TypeError):
                    # We might send in some invalid input accidentally.
                    # Ignore those inputs.
                    break

            # Only add this row if the for loop quit without break.
            else:
                # Convert to milliseconds
                row = [(mean * 1000, stddev * 1000) for mean, stddev in row]
                # Make the lowest value bold.
                min_indx = min(enumerate(row), key=lambda x: x[1])[0]
                row = ['{:.3f} ± {:.3f}'.format(*x) for x in row]
                row[min_indx] = self.bold(row[min_indx])
                table.add_row(value_label, *row)

        # Show the results in a table.
        display(Markdown(str(table)))

    @staticmethod
    def mean(x):
        return math.fsum(x) / len(x)

    @staticmethod
    def stddev(x):
        mean = Timer.mean(x)
        sum_of_squares = math.fsum((v - mean)**2 for v in x)
        return math.sqrt(sum_of_squares / (len(x) - 1))

    @staticmethod
    def bold(x):
        return "**{}**".format(x)
    
    def _timeit(self, call, setup, repeat=5):
        """Perform the actual timing and return a formatted string of the runtime"""
        result = timeit.repeat(call, setup, number=100000, repeat=repeat)
        return self.mean(result), self.stddev(result)

class Table(list):
    """List of strings that can be made into a Markdown table."""
    def add_row(self, *elements):
        self.append('|'.join(elements))
    def add_header(self, *elements):
        self.add_row(*elements)
        seperators = ['---'] * len(elements)
        seperators = [sep + (':' if i != 0 else '') for i, sep in enumerate(seperators)]
        self.add_row(*seperators)
    def __str__(self):
        return '\n'.join(self)


## Built-in Functions Drop-in Replacement Timing Results
The following timing tests compare the performance of Python's builtin `int` and `float` functions against the implementations from `fastnumbers` for various input types.

In [2]:
timer = Timer('Timing comparison of `int` functions')
timer.add_function('int', 'builtin')
timer.add_function('int', 'fastnumbers', 'from fastnumbers import int')
timer.time_functions(repeat=100)

### Timing comparison of `int` functions

Input type|builtin (ms)|fastnumbers (ms)
---|---:|---:
Int String|38.683 ± 4.786|**25.904 ± 3.699**
Large Int String|42.917 ± 3.750|**38.772 ± 3.171**
Int|18.586 ± 2.567|**15.980 ± 1.939**
Float|30.487 ± 2.989|**27.916 ± 2.829**

In [3]:
timer = Timer('Timing comparison of `float` functions')
timer.add_function('float', 'builtin')
timer.add_function('float', 'fastnumbers', 'from fastnumbers import float')
timer.time_functions(repeat=100)

### Timing comparison of `float` functions

Input type|builtin (ms)|fastnumbers (ms)
---|---:|---:
Int String|25.528 ± 3.027|**21.709 ± 2.300**
Large Int String|56.576 ± 7.385|**55.016 ± 4.036**
Float String|54.128 ± 3.977|**23.068 ± 2.203**
Large Float String|90.642 ± 4.793|**89.915 ± 5.405**
Int|19.447 ± 2.292|**16.192 ± 1.904**
Float|17.642 ± 1.728|**14.697 ± 1.746**

## Error-Handling Conversion Functions Timing Results
The following timing tests compare the performance of the `fastnumbers` functions that convert input to numeric types while doing error handling with common equivalent pure-Python implementations.

In [4]:
def int_re(x, int_match=re.compile(r'[-+]?\d+$').match):
    """Function to simulate fast_int but with regular expressions."""
    try:
        if int_match(x):
            return int(x)
        else:
            return x
    except TypeError:
        return int(x)

def int_try(x):
    """Function to simulate fast_int but with try/except."""
    try:
        return int(x)
    except ValueError:
        return x

timer = Timer('Timing comparison of `int` functions with error handling')
timer.add_function('int_try', 'try/except', 'from __main__ import int_try')
timer.add_function('int_re', 'regex', 'from __main__ import int_re')
timer.add_function('fast_int', 'fastnumbers', 'from fastnumbers import fast_int')
timer.time_functions()

### Timing comparison of `int` functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|306.610 ± 15.120|65.533 ± 4.574|**31.800 ± 2.661**
Int String|57.323 ± 1.871|174.989 ± 5.777|**26.233 ± 1.706**
Large Int String|74.986 ± 6.237|251.928 ± 23.637|**38.481 ± 1.581**
Float String|388.777 ± 35.151|132.290 ± 11.349|**22.959 ± 3.508**
Large Float String|368.133 ± 38.748|101.808 ± 4.850|**22.424 ± 1.436**
Int|34.597 ± 3.774|211.696 ± 11.107|**17.800 ± 2.528**
Float|46.020 ± 2.988|238.819 ± 6.450|**29.072 ± 2.738**

In [5]:
def float_re(x, float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate fast_float but with regular expressions."""
    try:
        if float_match(x):
            return float(x)
        else:
            return x
    except TypeError:
        return float(x)

def float_try(x):
    """Function to simulate fast_float but with try/except."""
    try:
        return float(x)
    except ValueError:
        return x

timer = Timer('Timing comparison of `float` functions with error handling')
timer.add_function('float_try', 'try/except', 'from __main__ import float_try')
timer.add_function('float_re', 'regex', 'from __main__ import float_re')
timer.add_function('fast_float', 'fastnumbers', 'from fastnumbers import fast_float')
timer.time_functions()

### Timing comparison of `float` functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|171.007 ± 12.830|82.048 ± 7.516|**31.797 ± 3.574**
Int String|43.924 ± 4.323|170.496 ± 6.504|**22.225 ± 1.587**
Large Int String|97.029 ± 7.698|299.396 ± 25.097|**72.344 ± 2.299**
Float String|88.048 ± 14.065|314.369 ± 17.788|**31.932 ± 1.355**
Large Float String|148.599 ± 9.293|383.095 ± 50.301|**97.033 ± 6.793**
Int|30.403 ± 0.228|203.575 ± 4.020|**17.463 ± 2.424**
Float|31.551 ± 4.304|193.288 ± 4.533|**15.566 ± 0.738**

In [6]:
def real_re(x,
            int_match=re.compile(r'[-+]?\d+$').match,
            real_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate fast_real but with regular expressions."""
    try:
        if int_match(x):
            return int(x)
        elif real_match(x):
            return float(x)
        else:
            return x
    except TypeError:
        if type(x) in (float, int):
            return x
        else:
            raise TypeError

def real_try(x):
    """Function to simulate fast_real but with try/except."""
    try:
        a = float(x)
    except ValueError:
        return x
    else:
        b = int(a)
        return b if a == b else b

timer = Timer('Timing comparison of `float` (but coerce to `int` if possible) functions with error handling')
timer.add_function('real_try', 'try/except', 'from __main__ import real_try')
timer.add_function('real_re', 'regex', 'from __main__ import real_re')
timer.add_function('fast_real', 'fastnumbers', 'from fastnumbers import fast_real')
timer.time_functions()

### Timing comparison of `float` (but coerce to `int` if possible) functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|174.234 ± 16.831|144.666 ± 22.226|**38.569 ± 1.305**
Int String|102.788 ± 12.355|172.778 ± 7.241|**29.357 ± 2.663**
Large Int String|186.016 ± 12.993|208.440 ± 16.758|**46.271 ± 10.849**
Float String|183.611 ± 39.832|313.941 ± 10.264|**41.157 ± 3.934**
Large Float String|228.257 ± 8.647|409.705 ± 35.163|**128.405 ± 7.650**
Int|78.695 ± 8.619|216.332 ± 6.986|**17.050 ± 2.046**
Float|98.338 ± 8.578|206.912 ± 5.944|**66.213 ± 6.611**

In [7]:
def forceint_re(x,
                int_match=re.compile(r'[-+]\d+$').match,
                float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate fast_forceint but with regular expressions."""
    try:
        if int_match(x):
            return int(x)
        elif float_match(x):
            return int(float(x))
        else:
            return x
    except TypeError:
        return int(x)

def forceint_try(x):
    """Function to simulate fast_forceint but with try/except."""
    try:
        return int(x)
    except ValueError:
        try:
            return int(float(x))
        except ValueError:
            return x

timer = Timer('Timing comparison of forced `int` functions with error handling')
timer.add_function('forceint_try', 'try/except', 'from __main__ import forceint_try')
timer.add_function('forceint_re', 'regex', 'from __main__ import forceint_re')
timer.add_function('fast_forceint', 'fastnumbers', 'from fastnumbers import fast_forceint')
timer.time_functions()

### Timing comparison of forced `int` functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|560.221 ± 47.464|127.134 ± 10.715|**32.451 ± 2.850**
Int String|44.128 ± 2.189|154.620 ± 18.279|**27.545 ± 2.340**
Large Int String|72.995 ± 4.475|375.822 ± 36.497|**45.901 ± 3.582**
Float String|487.252 ± 56.398|431.006 ± 45.451|**39.462 ± 5.209**
Large Float String|522.511 ± 13.213|410.493 ± 18.272|**125.913 ± 7.826**
Int|32.404 ± 1.096|203.569 ± 6.169|**15.287 ± 0.190**
Float|43.754 ± 0.391|257.235 ± 21.026|**32.738 ± 0.891**

## Checking Functions Timing Results
The following timing tests compare the performance of the `fastnumbers` functions that check if an input *could* be converted to numeric type with common equivalent pure-Python implementations.

In [8]:
def isint_re(x, int_match=re.compile(r'[-+]?\d+$').match):
    """Function to simulate isint but with regular expressions."""
    t = type(x)
    return t == int if t in (float, int) else bool(int_match(x))

def isint_try(x):
    """Function to simulate isint but with try/except."""
    try:
        int(x)
    except ValueError:
        return False
    else:
        return type(x) != float

timer = Timer('Timing comparison to check if value can be converted to `int`')
timer.add_function('isint_try', 'try/except', 'from __main__ import isint_try')
timer.add_function('isint_re', 'regex', 'from __main__ import isint_re')
timer.add_function('isint', 'fastnumbers', 'from fastnumbers import isint')
timer.time_functions()

### Timing comparison to check if value can be converted to `int`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|361.171 ± 40.095|180.444 ± 14.906|**36.127 ± 1.848**
Int String|85.313 ± 6.132|200.590 ± 15.140|**18.310 ± 0.121**
Large Int String|92.165 ± 8.389|225.181 ± 38.763|**23.068 ± 0.545**
Float String|304.545 ± 15.010|159.231 ± 9.885|**21.160 ± 2.213**
Large Float String|339.637 ± 33.684|191.718 ± 2.175|**22.326 ± 2.006**
Int|71.059 ± 2.954|64.223 ± 1.758|**18.519 ± 1.927**
Float|72.134 ± 7.363|49.557 ± 3.306|**15.739 ± 2.417**

In [9]:
def isfloat_re(x, float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate isfloat but with regular expressions."""
    t = type(x)
    return t == float if t in (float, int) else bool(float_match(x))

def isfloat_try(x):
    """Function to simulate isfloat but with try/except."""
    try:
        float(x)
    except ValueError:
        return False
    else:
        return type(x) != int

timer = Timer('Timing comparison to check if value can be converted to `float`')
timer.add_function('isfloat_try', 'try/except', 'from __main__ import isfloat_try')
timer.add_function('isfloat_re', 'regex', 'from __main__ import isfloat_re')
timer.add_function('isfloat', 'fastnumbers', 'from fastnumbers import isfloat')
timer.time_functions()

### Timing comparison to check if value can be converted to `float`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|185.556 ± 10.219|183.455 ± 22.155|**34.445 ± 4.255**
Int String|87.071 ± 8.170|188.349 ± 17.396|**24.171 ± 1.196**
Large Int String|103.048 ± 6.138|204.514 ± 5.878|**25.006 ± 4.774**
Float String|101.549 ± 7.032|194.083 ± 15.804|**25.625 ± 4.614**
Large Float String|180.110 ± 15.954|229.581 ± 25.109|**28.541 ± 1.852**
Int|79.042 ± 1.392|65.107 ± 5.561|**18.556 ± 1.964**
Float|54.582 ± 2.302|55.099 ± 4.466|**19.332 ± 3.137**

In [10]:
def isreal_re(x, real_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate isreal but with regular expressions."""
    return type(x) in (float, int) or bool(real_match(x))

def isreal_try(x):
    """Function to simulate isreal but with try/except."""
    try:
        float(x)
    except ValueError:
        return False
    else:
        return True

timer = Timer('Timing comparison to check if value can be converted to `float` or `int`')
timer.add_function('isreal_try', 'try/except', 'from __main__ import isreal_try')
timer.add_function('isreal_re', 'regex', 'from __main__ import isreal_re')
timer.add_function('isreal', 'fastnumbers', 'from fastnumbers import isreal')
timer.time_functions()

### Timing comparison to check if value can be converted to `float` or `int`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|214.230 ± 24.193|181.822 ± 18.835|**33.603 ± 3.227**
Int String|73.229 ± 17.554|199.743 ± 23.166|**22.986 ± 2.665**
Large Int String|96.568 ± 9.793|225.804 ± 19.315|**22.273 ± 2.483**
Float String|82.286 ± 18.051|192.765 ± 17.354|**30.791 ± 5.810**
Large Float String|118.216 ± 7.965|196.539 ± 7.806|**22.497 ± 4.458**
Int|42.778 ± 8.550|44.568 ± 2.864|**17.031 ± 3.027**
Float|31.371 ± 2.434|41.606 ± 2.312|**16.484 ± 2.091**

In [11]:
def isintlike_re(x,
                 int_match=re.compile(r'[-+]?\d+$').match,
                 float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate isintlike but with regular expressions."""
    try:
        if int_match(x):
            return True
        elif float_match(x):
            return float(x).is_integer()
        else:
            return False
    except TypeError:
        return int(x) == x

def isintlike_try(x):
    """Function to simulate isintlike but with try/except."""
    try:
        a = int(x)
    except ValueError:
        try:
            a = float(x)
        except ValueError:
            return False
        else:
            return a.is_integer()
    else:
        return a == float(x)

timer = Timer('Timing comparison to check if value can be coerced losslessly to `int`')
timer.add_function('isintlike_try', 'try/except', 'from __main__ import isintlike_try')
timer.add_function('isintlike_re', 'regex', 'from __main__ import isintlike_re')
timer.add_function('isintlike', 'fastnumbers', 'from fastnumbers import isintlike')
timer.time_functions()

### Timing comparison to check if value can be coerced losslessly to `int`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|515.784 ± 46.862|120.094 ± 4.339|**33.689 ± 2.491**
Int String|97.011 ± 5.278|92.303 ± 19.097|**18.526 ± 0.104**
Large Int String|159.635 ± 12.521|132.939 ± 17.332|**24.887 ± 1.149**
Float String|558.477 ± 29.132|380.323 ± 34.313|**19.436 ± 0.105**
Large Float String|586.478 ± 127.362|493.231 ± 10.817|**24.632 ± 0.341**
Int|77.774 ± 0.909|260.053 ± 31.371|**20.246 ± 3.934**
Float|121.280 ± 26.672|280.598 ± 8.459|**43.906 ± 3.013**